<a href="https://colab.research.google.com/github/Scyzentraz/modulajar/blob/main/Stable%20Upscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Langkah 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Langkah2. Install Real Esrgan & Dependensi
import os

if not os.path.exists("Real-ESRGAN"):
    !git clone https://github.com/xinntao/Real-ESRGAN.git

%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 759 (delta 106), reused 99 (delta 99), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 4.61 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using

In [ ]:
#Langkah 3. Patch bug rgb to grayscale (opsional)
file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"

with open(file_path, 'r') as file:
    file_data = file.readlines()

# Hanya patch jika belum dipatch
patched = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        patched = True
        break

if patched:
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("✅ Bug torchvision berhasil dipatch.")
else:
    print("✅ Tidak perlu patch, sudah sesuai.")

✅ Bug torchvision berhasil dipatch.


In [ ]:
#Langkah 4.1 siapkan file input
#Langkah 4.2 konfigurasi batch
import os
from glob import glob

# Konfigurasi
source = "image"
input_dir = "/content/drive/MyDrive/upscale/input"
output_dir = "/content/drive/MyDrive/upscale/output"
model = "RealESRGAN_x2plus"
scale = 2
face_enhance = False

# Buat folder output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Ambil semua file jpg/jpeg
image_paths = sorted(
    glob(os.path.join(input_dir, "*.jpg")) +
    glob(os.path.join(input_dir, "*.jpeg")) +
    glob(os.path.join(input_dir, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")

Jumlah gambar ditemukan: 1


In [ ]:
#Langkah 5.1
#upscale tahap 1
from tqdm import tqdm

code = "inference_realesrgan.py"
tile = 400  # ukuran tile, bisa disesuaikan jika masih kehabisan memori

# Jalankan batch dengan progress bar
for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.basename(input_path)
    output_path = os.path.join(output_dir, filename)

    command = f'python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --tile {tile}'

    if face_enhance:
        command += " --face_enhance"

    os.system(command)

#Setelah gambar berhasil, kembali ke langkah 4.1, tetapi kini folder input diisi dari gambar upscale tahap pertama (diambil dari folder output tadi) kemudian lanjut lagi ke langkah 4.2, lalu ke langkah 5.2

🚀 Memproses gambar: 100%|██████████| 1/1 [00:09<00:00,  9.75s/it]


In [ ]:
#Langkah 5.2
#upscale tahap 2
from tqdm import tqdm
import os

code = "inference_realesrgan.py"
tile = 400  # ukuran tile, bisa disesuaikan kalau masih kehabisan memori

# Jalankan batch dengan progress bar
for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_dir}" -s {scale} --tile {tile}'

    if face_enhance:
        command += " --face_enhance"

    print("\n[📤 Menjalankan command:]")
    print(command)  # log command yang dijalankan

    exit_code = os.system(command)

    if exit_code != 0:
        print(f"[❌ Gagal memproses {input_path}] (exit code: {exit_code})")
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

#Setelah gambar berhasil, kembali ke langkah 4.1, kini folder input diisi dari gambar upscale tahap kedua (diambil dari folder output tadi) kemudian lanjut lagi ke langkah 4.2, lalu ke langkah 5.3

🚀 Memproses gambar:   0%|          | 0/1 [00:00<?, ?it/s]


[📤 Menjalankan command:]
python inference_realesrgan.py -i "/content/drive/MyDrive/upscale/input/20250630_095343.jpg" -n RealESRGAN_x2plus -o "/content/drive/MyDrive/upscale/output" -s 2 --tile 400


🚀 Memproses gambar: 100%|██████████| 1/1 [00:09<00:00,  9.02s/it]

[✅ Berhasil memproses /content/drive/MyDrive/upscale/input/20250630_095343.jpg]


In [ ]:
#Langkah 5.3
#upscale tahap 3
from tqdm import tqdm
import subprocess
import os

tile = 0  #
tile_pad = 10
code = "inference_realesrgan.py"

for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    command = f'python {code} -i "{input_path}" -n {model} -o "{output_dir}" -s {scale} --tile {tile} --tile_pad {tile_pad}'

    if face_enhance:
        command += " --face_enhance"

    print("[📤 Menjalankan command:]")
    print(command)

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"[❌ Gagal memproses {input_path}]")
        print("STDERR:", result.stderr)
    else:
        print(f"[✅ Berhasil memproses {input_path}]")

🚀 Memproses gambar:   0%|          | 0/1 [00:00<?, ?it/s]

[📤 Menjalankan command:]
python inference_realesrgan.py -i "/content/drive/MyDrive/upscale/input/20250630_095343.jpg" -n RealESRGAN_x2plus -o "/content/drive/MyDrive/upscale/output" -s 2 --tile 0 --tile_pad 10


🚀 Memproses gambar: 100%|██████████| 1/1 [00:08<00:00,  8.65s/it]

[✅ Berhasil memproses /content/drive/MyDrive/upscale/input/20250630_095343.jpg]
